<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Machine Learning Para Aplicações Biomédicas</font>
## <font color='blue'>Projeto 2</font>
### <font color='blue'>Análise de Dados de Exames Médicos Para Previsão de Doenças com Modelo XGBoost</font>

## Pacotes Python Usados no Projeto

In [1]:
!pip install -q -U watermark

O procedimento de instalação de pacotes e dependências está no arquivo LEIAME.txt

In [2]:
#!pip install -q xgboost

In [3]:
# Import
import pickle
import sklearn as sk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, roc_curve
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Etapa 1 - Definindo e Compreendendo o Problema a Ser Resolvido

Objetivo: Prever se um paciente está tendo uma convulsão ou não através de 178 leituras de EEG (Eletroencefalograma) por segundo.

## Etapa 2 - Compreensão dos Dados

O conjunto de dados inclui 4097 leituras de eletroencefalograma (EEG) por paciente durante 23,5 segundos, com 500 pacientes no total. Os 4097 pontos de dados foram então divididos igualmente em 23 partes por paciente, cada parte é convertida em uma linha no conjunto de dados. Cada linha contém 178 leituras, que são transformadas em colunas; em outras palavras, existem 178 colunas que compõem um segundo das leituras de EEG. No total, existem 11.500 linhas e 179 colunas com a última coluna contendo o status do paciente, esteja ele tendo uma convulsão ou não.

## Etapa 3 - Carregando os Dados

In [5]:
# Carregando os dados
df_dsa = pd.read_csv("dados/dados_originais.csv")

In [6]:
# Shape
df_dsa.shape

(11500, 180)

In [7]:
# Visualizando alguns registros
df_dsa.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


## Etapa 4 - Análise Exploratória e Definição da Variável Alvo

Vamos criar uma coluna chamada LABEL_VARIAVEL_TARGET em que 1 é quando um paciente está tendo uma convulsão e 0 é quando um paciente não está tendo uma convulsão.

Na última coluna, somente o valor 1 representa convulsão. Os demais valores não representam convulsão.

In [8]:
# Colocando True onde o valor for igual a 1 e False onde o valor for diferente.
df_dsa["LABEL_VARIAVEL_TARGET"] = df_dsa.y == 1

In [9]:
# Visualizando alguns registros
df_dsa.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X171,X172,X173,X174,X175,X176,X177,X178,y,LABEL_VARIAVEL_TARGET
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-15,-31,-77,-103,-127,-116,-83,-51,4,False
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,150,146,152,157,156,154,143,129,1,True
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,64,48,19,-12,-30,-35,-35,-36,5,False
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-81,-80,-77,-85,-77,-72,-69,-65,5,False
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,2,-12,-32,-41,-65,-83,-89,-73,5,False


In [10]:
# Ajusta o tipo para int
df_dsa["LABEL_VARIAVEL_TARGET"] = df_dsa["LABEL_VARIAVEL_TARGET"].astype(int)

In [11]:
# Visualizando alguns registros
df_dsa.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X171,X172,X173,X174,X175,X176,X177,X178,y,LABEL_VARIAVEL_TARGET
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-15,-31,-77,-103,-127,-116,-83,-51,4,0
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,150,146,152,157,156,154,143,129,1,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,64,48,19,-12,-30,-35,-35,-36,5,0
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-81,-80,-77,-85,-77,-72,-69,-65,5,0
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,2,-12,-32,-41,-65,-83,-89,-73,5,0


In [12]:
# A coluna original (y) que continha se um paciente está tendo uma convulsão será eliminada, pois era uma variável 
# categórica com 5 status diferentes. Desde então, convertemos isso em uma variável numérica binária 
# chamada LABEL_VARIAVEL_TARGET.
df_dsa.pop('y')

0        4
1        1
2        5
3        5
4        5
        ..
11495    2
11496    1
11497    5
11498    3
11499    4
Name: y, Length: 11500, dtype: int64

In [13]:
# Visualizando alguns registros
df_dsa.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,LABEL_VARIAVEL_TARGET
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [14]:
# A primeira coluna será descartada devido à sua inutilidade em nosso modelo de aprendizado de máquina. 
# Não precisamos do ID
df_dsa.drop(df_dsa.columns[0], axis = 1, inplace = True)

In [15]:
# Visualizando alguns registros
df_dsa.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,LABEL_VARIAVEL_TARGET
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [16]:
print("Número de colunas:", len(df_dsa.columns))

Número de colunas: 179


In [17]:
# Verificando valores ausentes
df_dsa.isna().sum().sum()

np.int64(0)

As colunas foram divididas para capturar a leitura do EEG em um ponto no tempo e todos os pontos no tempo (todas as 178 colunas) existem no mesmo segundo. Cada linha tem as 178 colunas com as medidas, mais a coluna indicando se as medidas são de convulsão ou não.

## Etapa 5 - Calculando a Prevalência da Classe Positiva

A prevalência é a porcentagem de amostras que tem a característica que você está tentando prever. Nesse cenário específico, significa que as pessoas que têm uma convulsão representam a classe positiva (ocorrência do evento), enquanto as que não sofrem convulsão representam a classe negativa (não ocorrência do evento). 

A taxa é calculada por (número de amostras positivas / número de amostras). Portanto, uma taxa de prevalência de 0,2 significa que 20% de nossa amostra são de pacientes que tiveram convulsão.

In [18]:
# Esta função calcula a prevalência da classe positiva (label = 1)
def calcula_prevalencia(y_actual):
    return sum(y_actual) / len(y_actual)

In [19]:
print("Prevalência da classe positiva: %.3f"% calcula_prevalencia(df_dsa["LABEL_VARIAVEL_TARGET"].values))

Prevalência da classe positiva: 0.200


## Etapa 6 - Limpeza dos Dados

Para o conjunto de dados da epilepsia, existem 178 recursos (colunas), no entanto, uma vez que cada coluna representa um ponto de dados em um ponto específico no tempo e são todas as leituras de EEG, não há necessidade de realizar transformação adicional.

In [20]:
# Preparando o dataset somente com os dados de interesse
collist = df_dsa.columns.tolist()
cols_input = collist[0:178]
df_data = df_dsa[cols_input + ["LABEL_VARIAVEL_TARGET"]]

In [21]:
df_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,LABEL_VARIAVEL_TARGET
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [22]:
# Checando se temos colunas duplicadas nos dados de entrada
dup_cols = set([x for x in cols_input if cols_input.count(x) > 1])
print(dup_cols)
assert len(dup_cols) == 0, "você duplicou colunas em cols_input"

set()


In [23]:
# Checando se temos colunas duplicadas no dataset final
cols_df_data = list(df_data.columns)
dup_cols = set([x for x in cols_df_data if cols_df_data.count(x) > 1])
print(dup_cols)
assert len(dup_cols) == 0,'você duplicou colunas em df_data'

set()


## Etapa 7 - Divisão dos Dados em Treino, Validação e Teste

Geralmente, podemos dividir o conjunto de dados em 50/25/25, 60/20/20, 70/15/15 como a divisão para amostras de treinamento / validação / teste; isso também depende de quantas amostras temos. Se tivermos um conjunto de dados extremamente grande (centenas de milhões de linhas), podemos usar uma divisão como 98/1/1. 

A divisão de treinamento é usada para treinar nosso algoritmo de aprendizado de máquina, por isso queremos usar a maioria de nosso conjunto de dados. O conjunto de dados de validação é usado para ajustar os hiperparâmetros e selecionar a abordagem de melhor desempenho. O conjunto de dados de teste é usado para testar a precisão do nosso modelo de aprendizado de máquina.

Leia o material no Capítulo 6 sobre Data Leakage.

In [24]:
# Gerando amostras aleatórias dos dados
df_data = df_data.sample(n = len(df_data))

In [25]:
df_data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,LABEL_VARIAVEL_TARGET
5562,-22,-59,-81,-86,-81,-108,-84,-93,-98,-128,...,59,55,84,168,295,325,307,269,232,0
7243,-420,-672,-903,-1145,-992,-678,-100,462,848,1057,...,-144,-138,-157,-186,-216,-199,-155,-57,-1,1
6048,41,42,30,15,4,-14,-33,-46,-54,-63,...,15,1,-12,-9,2,14,15,2,-6,0
3879,2,5,3,-7,-11,-19,-6,-2,-2,2,...,-1,0,-9,-17,-12,-9,-6,0,1,0
919,-57,-49,-51,-65,-77,-93,-108,-132,-138,-140,...,-45,-37,-41,-48,-68,-81,-81,-79,-80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,-94,-76,-60,-28,-17,-21,-44,-48,-30,5,...,-31,-42,-60,-55,-35,4,35,46,42,0
719,-38,-21,-5,2,24,32,35,28,11,0,...,-71,-79,-63,-41,-12,10,-1,-31,-47,0
7624,40,52,57,45,37,15,5,1,-10,-18,...,-20,-24,-23,-12,2,30,55,63,69,0
4465,-157,-103,-45,10,64,113,152,184,205,219,...,-17,23,54,79,100,110,119,123,122,1


In [26]:
# Ajustando os índices do dataset
df_data = df_data.reset_index(drop = True)

In [27]:
# Gera um índice para a divisão (30% dos dados vão para validação/teste, ou seja, faremos divisão 70/15/15)
df_valid_teste = df_data.sample(frac = 0.3)
print("Tamanho da divisão de validação / teste: %.1f" % (len(df_valid_teste) / len(df_data)))

Tamanho da divisão de validação / teste: 0.3


In [28]:
# Linhas que serão distribuídas entre validação e teste
df_valid_teste

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,LABEL_VARIAVEL_TARGET
6345,-3,40,65,69,57,39,19,8,11,15,...,40,78,107,113,98,71,50,33,5,0
9300,57,53,42,37,32,21,9,-6,-18,-28,...,-37,-37,-40,-41,-45,-54,-66,-73,-74,0
8978,-68,-19,24,65,100,123,147,156,159,159,...,-7,22,47,70,94,109,118,123,121,1
2888,246,203,156,76,-42,-161,-214,-193,-87,14,...,441,442,399,384,397,395,364,318,268,1
2397,-63,-65,-69,-76,-84,-88,-97,-99,-97,-88,...,-50,-48,-45,-36,-33,-34,-32,-30,-38,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,60,65,62,64,54,42,18,9,-12,-18,...,14,1,-25,-50,-58,-62,-46,-58,-57,0
1619,556,279,93,-50,-173,-256,-311,-446,-536,-472,...,312,57,-331,-346,-422,-169,-6,37,-23,1
10617,207,214,216,217,219,226,225,223,226,226,...,-65,-62,-59,-60,-69,-64,-68,-58,-53,0
4742,-19,-30,-36,-35,-25,-11,0,-11,-8,-20,...,19,0,-3,-1,1,20,12,4,-12,0


In [29]:
# Dados de teste
df_teste = df_valid_teste.sample(frac = 0.5)
len(df_teste)

1725

In [30]:
# Dados se validação
df_valid = df_valid_teste.drop(df_teste.index)
len(df_valid)

1725

In [31]:
# Dados de treino
df_treino = df_data.drop(df_valid_teste.index)
len(df_treino)

8050

In [32]:
# Verifica se as amostras somadas totalizam o volume original dos dados
print('Todas as amostras (n = %d)'%len(df_data))
assert len(df_data) == (len(df_teste) + len(df_valid) + len(df_treino)), 'algo saiu errado'

Todas as amostras (n = 11500)


In [33]:
# Verifica a prevalência da classe positiva em cada subconjunto
print("Teste(n = %d): %.3f" % (len(df_teste), calcula_prevalencia(df_teste.LABEL_VARIAVEL_TARGET.values)))
print("Validação(n = %d): %.3f" % (len(df_valid), calcula_prevalencia(df_valid.LABEL_VARIAVEL_TARGET.values)))
print("Treino(n = %d): %.3f" % (len(df_treino), calcula_prevalencia(df_treino.LABEL_VARIAVEL_TARGET.values)))

Teste(n = 1725): 0.207
Validação(n = 1725): 0.196
Treino(n = 8050): 0.199


## Etapa 8 - Balanceamento de Classe

Leia o material no Capítulo 6 sobre os impactos do Balanceamento de Classe.

In [34]:
# Proporção de classes em treino antes do balanceamento de classe
df_treino.LABEL_VARIAVEL_TARGET.value_counts()

LABEL_VARIAVEL_TARGET
0    6445
1    1605
Name: count, dtype: int64

In [35]:
# Cria um índice
rows_pos = df_treino.LABEL_VARIAVEL_TARGET == 1

In [36]:
# Define valores positivos e negativos do índice
df_train_pos = df_treino.loc[rows_pos]
df_train_neg = df_treino.loc[~rows_pos]

In [37]:
# Valor mínimo
n = np.min([len(df_train_pos), len(df_train_neg)])

In [38]:
# Obtém valores aleatórios para o dataset de treino
df_treino_final = pd.concat([df_train_pos.sample(n = n, random_state = 64), 
                             df_train_neg.sample(n = n, random_state = 64)], 
                            axis = 0, 
                            ignore_index = True)

In [39]:
# Amostragem
df_treino_final = df_treino_final.sample(n = len(df_treino_final), random_state = 64).reset_index(drop = True)

In [40]:
print('Balanceamento em Treino(n = %d): %.3f'%(len(df_treino_final), 
                                               calcula_prevalencia(df_treino_final.LABEL_VARIAVEL_TARGET.values)))

Balanceamento em Treino(n = 3210): 0.500


In [41]:
# Proporção de classes em treino depois do balanceamento de classe
df_treino_final.LABEL_VARIAVEL_TARGET.value_counts()

LABEL_VARIAVEL_TARGET
1    1605
0    1605
Name: count, dtype: int64

In [42]:
# Salvamos todos os datasets em disco no formato csv.
df_treino.to_csv('dados/dados_treino.csv', index = False)
df_treino_final.to_csv('dados/dados_treino_final.csv', index = False)
df_valid.to_csv('dados/dados_valid.csv', index = False)
df_teste.to_csv('dados/dados_teste.csv', index = False)

In [43]:
# Salvamos os dados de entrada (colunas preditoras) para facilitar a utilização mais tarde
pickle.dump(cols_input, open('dados/cols_input.sav', 'wb'))

> Criamos as Matrizes X e Y.

In [44]:
# X
X_treino = df_treino_final[cols_input].values
X_valid = df_valid[cols_input].values

In [45]:
# Y
y_treino = df_treino_final['LABEL_VARIAVEL_TARGET'].values
y_valid = df_valid['LABEL_VARIAVEL_TARGET'].values

In [46]:
# Print
print('Shape dos dados de treino:', X_treino.shape, y_treino.shape)
print('Shape dos dados de validação:', X_valid.shape, y_valid.shape)

Shape dos dados de treino: (3210, 178) (3210,)
Shape dos dados de validação: (1725, 178) (1725,)


In [47]:
X_treino

array([[  57,   72,  122, ...,  102,   55,   -4],
       [-309, -299, -264, ..., -121, -126, -130],
       [  19,   22,   21, ...,   -1,   -9,  -12],
       ...,
       [ -83,  -65,  -59, ...,  -52,  -29,   -7],
       [ 124,   41,    3, ...,  206,  337,  364],
       [ -18,   -9,    9, ...,    4,   15,   12]], shape=(3210, 178))

## Etapa 9 - Padronização

Leia o material no Capítulo 6 sobre os quando aplicar Padronização.

In [48]:
# Cria o objeto
scaler = StandardScaler()

In [49]:
# Faz o fit
scaler.fit(X_treino)

,copy,True
,with_mean,True
,with_std,True


In [50]:
# Salva o objeto em disco e carrega para usamos adiante
scalerfile = 'dados/scaler.sav'

In [51]:
pickle.dump(scaler, open(scalerfile, 'wb'))
scaler = pickle.load(open(scalerfile, 'rb'))

In [52]:
# Aplica a normalização em nossas matrizes de dados
X_treino_tf = scaler.transform(X_treino)
X_valid_tf = scaler.transform(X_valid)

In [53]:
X_treino_tf

array([[ 0.28511087,  0.33888431,  0.54757793, ...,  0.50172121,
         0.31546527,  0.07463751],
       [-1.16255535, -1.13931962, -1.04078124, ..., -0.43192137,
        -0.4430849 , -0.44509952],
       [ 0.13480672,  0.13966545,  0.131971  , ...,  0.07048719,
         0.04724864,  0.04163833],
       ...,
       [-0.26864124, -0.20697536, -0.19722261, ..., -0.14303645,
        -0.03656906,  0.06226282],
       [ 0.5501208 ,  0.21536862,  0.05790244, ...,  0.93714196,
         1.49729481,  1.59259963],
       [-0.01154205,  0.01614976,  0.08259196, ...,  0.09142088,
         0.14782987,  0.14063586]], shape=(3210, 178))

# Continuaremos no Próximo Capítulo.

In [54]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [55]:
#%watermark -v -m

In [56]:
#%watermark --iversions

# Fim